In [1]:
# Imports
import pathlib
import os
from dataclasses import dataclass, field
from typing import Dict, Literal, Optional, Tuple
import json

import imutils
import numpy as np
import cv2
from dataclasses_json import DataClassJsonMixin
import pandas as pd
from tqdm import tqdm

# Constants 
CWD = pathlib.Path(os.path.abspath(""))
GIT_ROOT = CWD.parent
DATA_DIR = GIT_ROOT / "data" / 'PhotosynthesisFall2022'

In [2]:
# Game state reconstruction routines
@dataclass
class Participant(DataClassJsonMixin):
    id: str
    position: Tuple[float, float]
    state: Literal['null', 'H2O', 'CO2', 'Sugar', 'O2', 'Thinking_H2O'] = 'null'

@dataclass
class EnvironmentState(DataClassJsonMixin):
    sun_state: Optional[bool] = None

@dataclass
class GameState(DataClassJsonMixin):
    participants: Dict[str, Participant] = field(default_factory=dict)
    environment: EnvironmentState = field(default_factory=EnvironmentState)

In [3]:
# Helper functions
def xy_transforms(xy: Tuple[float, float], w: int, h:int, c: Dict[str, float]) -> Tuple[float, float]:
    x, y = xy
    xx = int(((x+1)*h/2)*c['AFFINE'][0] + c['OFFSET'][0])
    yy = int(((y+1)*w/2)*c['AFFINE'][1] + c['OFFSET'][1])
    return (xx, yy)

def render(game_state: GameState, frame: np.ndarray, c: Dict[str, float]):

    if game_state.participants:
        for p in game_state.participants.values():
            p.position = (eval(p.position[0]), eval(p.position[1]))
            xy = xy_transforms(p.position, frame.shape[0], frame.shape[1], c)
            frame = cv2.circle(frame, xy, 30, (0, 0, 255), 1)
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (0,0,0), 
                2, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.id, 
                (xy[0]-22, xy[1]-11), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.4, 
                (255,255,255), 
                1, 
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (0,0,0), 
                2,
                cv2.LINE_AA
            )
            frame = cv2.putText(
                frame, 
                p.state, 
                (xy[0]-5*(len(p.state)), xy[1]+13), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6, 
                (255,255,255), 
                1,
                cv2.LINE_AA
            )

    return frame

In [4]:
# Transformation constants
START_INDEX = 0
# N = 20_000
RECORD = True

def time_alignment(id, vid_file, game_state_file, time_offset, correction):
    
    # Load the data
    assert vid_file.exists()
    cap = cv2.VideoCapture(str(vid_file))
    fps = cap.get(cv2.CAP_PROP_FPS)

    assert game_state_file.exists()
    game_state_logs = pd.read_csv(game_state_file)

    # Generate timestamp
    game_state_logs['datetime'] = pd.to_datetime(game_state_logs['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
    game_state_logs['timestamp'] = (game_state_logs['datetime'] - game_state_logs['datetime'].iloc[0]).dt.total_seconds()

    # If you want to save the video
    if RECORD:
        video_file = DATA_DIR / 'time_alignment' / f"{id}-alignment.mp4"
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
        out = cv2.VideoWriter(str(video_file),fourcc,fps,(650,566))

    # Reset video and logs
    cap.set(cv2.CAP_PROP_POS_FRAMES, START_INDEX)
    game_state_pointer = 0
    N = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in tqdm(range(N), total=N):
        
        ret, frame = cap.read()
        if not ret:
            break

        frame_timestamp = (START_INDEX + i) / fps
        frame = cv2.putText(
            frame, 
            f"{pd.Timestamp(frame_timestamp, unit='s').strftime('%H:%M:%S.%f')}", 
            (5,30), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            1, 
            (0,0,255), 
            1, 
            cv2.LINE_AA
        )

        while game_state_logs['timestamp'].iloc[game_state_pointer+1] < frame_timestamp + time_offset:
            game_state_pointer += 1
        game_state = GameState.from_json(game_state_logs.iloc[game_state_pointer].state)

        # For macro alignment
        frame = cv2.putText(
            frame, 
            f"{game_state_pointer}", 
            (5,60), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            1, 
            (0,0,255), 
            1, 
            cv2.LINE_AA
        )

        frame = render(game_state, frame, correction)
        cv2.imshow(f'{id}_frame', imutils.resize(frame, width=1000))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if RECORD:
            out.write(frame)

    cv2.destroyAllWindows()
    if RECORD:
        out.release()

    # Record meta data
    json_file = DATA_DIR/'time_alignment'/f'{id}_meta.json'
    json_data = {'time_offset': time_offset, 'corrections': correction}
    with open(json_file, 'w') as f:
        json.dump(json_data, f, indent=4)

    # Save the data with the timestamp data
    aligned_logs_file = DATA_DIR/'time_alignment'/f'{id}_aligned_game_state.csv'
    game_state_logs['timestamp'] = game_state_logs['timestamp'] - time_offset
    aligned_state_logs = game_state_logs[game_state_logs['timestamp'] >= 0]
    aligned_state_logs.timestamp = aligned_state_logs.timestamp.round(3)
    aligned_state_logs.to_csv(aligned_logs_file, index=False)

In [5]:
# Processing - Day 11
TIME_OFFSET = 792.26
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}
time_alignment(
    'day11', 
    DATA_DIR / 'videos' / 'day 11' / "day11-screen-recording.mp4",
    DATA_DIR / 'game_state' / 'day_11_game_state.csv',
    TIME_OFFSET,
    CORRECTIONS
)

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
 26%|██▋       | 4307/16351 [01:18<03:39, 54.78it/s]


In [6]:
# Processing Day 12
TIME_OFFSET = 959.75
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}
time_alignment(
    'day12', 
    DATA_DIR / 'videos' / 'day 12' / "day12-screen-recording.mp4",
    DATA_DIR / 'game_state' / 'day_12_game_state.csv',
    TIME_OFFSET,
    CORRECTIONS
)

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 19798/19798 [04:16<00:00, 77.24it/s]
/tmp/ipykernel_11155/2652217590.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aligned_state_logs.timestamp = aligned_state_logs.timestamp.round(3)


In [22]:
# Processing Day 13
TIME_OFFSET = 4*60 + 30.9
CORRECTIONS = {'OFFSET': (-252,-280), 'AFFINE': (1.8,2)}
time_alignment(
    'day13', 
    DATA_DIR / 'videos' / 'day 13' / "day13-screen-recording-corrected.mp4",
    DATA_DIR / 'game_state' / 'day_13_game_state.csv',
    TIME_OFFSET,
    CORRECTIONS
)

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100%|██████████| 20000/20000 [05:56<00:00, 56.11it/s]
/tmp/ipykernel_13049/2540892142.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aligned_state_logs.timestamp = aligned_state_logs.timestamp.round(3)
